In [1]:
import numpy as np
from numpy import shape, matrix, log, exp, zeros,random,dot,multiply
from mnist import readWithoutBias
from math import sqrt

In [2]:
data_train, label_train = readWithoutBias(dataset="training")
data_test, label_test = readWithoutBias(dataset="testing")
label_train = matrix(label_train)
label_test  = matrix(label_test)
data_train  = matrix(data_train).T
data_test   = matrix(data_test).T

In [3]:
train_mean  = data_train.mean(axis=1)
train_std   = data_train.std(axis=1)
data_train  = np.nan_to_num((data_train - train_mean)/train_std)
data_test   = np.nan_to_num((data_test - train_mean)/train_std)

/Users/apoorve/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: RuntimeWarning: invalid value encountered in divide
  app.launch_new_instance()
/Users/apoorve/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: RuntimeWarning: divide by zero encountered in divide
/Users/apoorve/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: RuntimeWarning: invalid value encountered in divide


In [4]:
X_test = data_test
y_test = label_test

train = zip(data_train.T,label_train)
mini_batch_size = 10

n_input = shape(data_train)[0] # excluding bias term
n_hidden = 30               # excluding bias term
n_output = 10
epochs = 10
alpha = 0.1
mini_batch_size = 10

##Initializations

In [8]:
random.seed(0)
theta1 = matrix(random.randn(n_hidden,n_input))/sqrt(n_input)
bias1  = matrix(random.randn(n_hidden,1))
theta2 = matrix(random.randn(n_output,n_hidden))/sqrt(n_hidden)
bias2  = matrix(random.randn(n_output,1))

afunc, afuncGradient = act_funcs["sigmoid"]

#Regularization term
lam = 0.0

#Momentum Term
gamma = 0.0
v1 = np.zeros_like(theta1)
vb1= np.zeros_like(bias1)
v2 = np.zeros_like(theta2)
vb2= np.zeros_like(bias2)

### Training Code

In [39]:
print "Layers:\t\t\t", 3
print "Hidden Nodes:\t\t",n_hidden
print "Epochs:\t\t\t", epochs
print "LearningRate:\t\t",alpha
print "Mini Batch Size:\t",mini_batch_size
print "Activation Fn:\t\ttanh"
print "Gamma(Momentum):\t",gamma
print "Lambda(Regularize):\t",lam

for i in range(epochs):
    random.shuffle(train)
    mini_batches = [train[k:k+mini_batch_size] for k in xrange(0,len(train),mini_batch_size)]
    for mini_batch in mini_batches:
        d1  = np.zeros_like(theta1)
        d2  = np.zeros_like(theta2)
        db1 = np.zeros_like(bias1)
        db2 = np.zeros_like(bias2)
        
        for X,y in mini_batch:
            gradTheta1, gradBias1, gradTheta2, gradBias2 = \
                backPropGradient(X.T, y, theta1, theta2, bias1, bias2) #just one example passed
            d1  += gradTheta1
            db1 += gradBias1
            d2  += gradTheta2
            db2 += gradBias2
            
        d1   = d1/mini_batch_size + lam*theta1
        db1  = db1/mini_batch_size
        d2   = d2/mini_batch_size + lam*theta2
        db2  = db2/mini_batch_size
        
        v1 = alpha*d1  + v1*gamma
        vb1= alpha*db1 + vb1*gamma
        v2 = alpha*d2  + v2*gamma
        vb2= alpha*db2 + vb2*gamma
        
        theta1 = theta1 - v1
        bias1  = bias1  - vb1
        theta2 = theta2 - v2
        bias2  = bias2  - vb2
    print "Epoch",i+1,"train,test accuracy:\t",accuracy(data_train, label_train, theta1, theta2, bias1, bias2), \
            accuracy(data_test, label_test , theta1, theta2, bias1, bias2)

Layers:			3
Hidden Nodes:		30
Epochs:			10
LearningRate:		0.1
Mini Batch Size:	10
Activation Fn:		tanh
Gamma(Momentum):	0.0
Lambda(Regularize):	0.0
Epoch 1 train,test accuracy:	0.929516666667 0.9219
Epoch 2 train,test accuracy:	0.9389 0.9274
Epoch 3 train,test accuracy:	0.944583333333 0.9344
Epoch 4 train,test accuracy:	0.946416666667 0.9338
Epoch 5 train,test accuracy:	0.951266666667 0.9341
Epoch 6 train,test accuracy:	0.953616666667 0.9374
Epoch 7 train,test accuracy:	0.954083333333 0.9348
Epoch 8 train,test accuracy:	0.956233333333 0.9364
Epoch 9 train,test accuracy:	0.958216666667 0.9373
Epoch 10 train,test accuracy:	0.958 0.9368


### Checking backpropgradient vs numerical gradient

In [14]:
X, y = train[0]
X = X.T
gradTheta1, gradBias1, gradTheta2, gradBias2 = backPropGradient(X,y,theta1,theta2, bias1, bias2)
numGrad1,numGradBias1,numGrad2,numGradBias2  = numericalGradient(X,y,theta1,theta2, bias1, bias2)
print np.sum(np.subtract(numGradBias2,gradBias2))
print np.sum(np.subtract(numGradBias1,gradBias1))
print np.sum(np.subtract(numGrad2,gradTheta2))
print np.sum(np.subtract(numGrad1,gradTheta1))

1.62795166236e-10
-1.45355906397e-09
-1.01694661481e-10
-8.43307289101e-09


In [16]:
print np.mean(np.subtract(numGradBias2,gradBias2))
print np.mean(np.subtract(numGradBias1,gradBias1))
print np.mean(np.subtract(numGrad2,gradTheta2))
print np.mean(np.subtract(numGrad1,gradTheta1))

1.62795166236e-11
-4.84519687989e-11
-3.38982204936e-13
-3.58549017475e-13


In [17]:
print np.max(np.subtract(numGradBias2,gradBias2))
print np.max(np.subtract(numGradBias1,gradBias1))
print np.max(np.subtract(numGrad2,gradTheta2))
print np.max(np.subtract(numGrad1,gradTheta1))

print np.min(np.subtract(numGradBias2,gradBias2))
print np.min(np.subtract(numGradBias1,gradBias1))
print np.min(np.subtract(numGrad2,gradTheta2))
print np.min(np.subtract(numGrad1,gradTheta1))

6.21693252434e-11
1.36737973808e-10
1.76029912868e-10
1.79982517778e-09
-6.05020478162e-11
-2.40977672712e-10
-1.39576128433e-10
-1.98751259894e-09


### Helper Functions

In [9]:
def accuracy(X, y, theta1, theta2, bias1, bias2):
    a1 = X
    z2 = dot(theta1,a1) + bias1
    a2 = afunc(z2)
    z3 = dot(theta2,a2) + bias2
    a3 = sigmoid(z3)
    
    pred = np.argmax(a3,axis=0)
    return np.sum(np.equal(pred,y.T))/float(len(y))

In [10]:
def backPropGradient(X, y, theta1, theta2, bias1, bias2):
    a1 = X
    z2 = dot(theta1,a1) + bias1
    a2 = afunc(z2)
    z3 = dot(theta2,a2) + bias2
    a3 = sigmoid(z3)
    
    t = np.zeros_like(a3)
    for i in range(len(y)):
        t[y[i],i] = 1
    
    delta3 = a3-t #shape (10,1)
    delta2 = dot(theta2.T,delta3)#shape (100,10)X(10,1) = (100,1)
    delta2 = multiply(delta2,afuncGradient(z2))

    gradTheta2 = dot(delta3,a2.T)
    gradBias2  = delta3
    gradTheta1 = dot(delta2,a1.T)
    gradBias1  = delta2
    return gradTheta1, gradBias1, gradTheta2, gradBias2

In [11]:
def errorFn(X, y, theta1, theta2, bias1, bias2):
    n_examples = shape(X)[1]
    a1 = X
    z2 = dot(theta1,a1) + bias1
    a2 = afunc(z2)
    z3 = dot(theta2,a2) + bias2
    a3 = sigmoid(z3)
    
    t = np.zeros_like(a3)
    for i in range(len(y)):
        t[y[i],i] = 1

    error = np.sum(multiply(t,log(a3)) + multiply((1-t),log(1-a3)))
    error = -error/n_examples
    return error

In [12]:
def numericalGradient(X, y, theta1, theta2, bias1, bias2):
    epsilon = 10**-5
    numGrad1     = np.zeros_like(theta1)
    numGradBias1 = np.zeros_like(bias1)
    numGrad2     = np.zeros_like(theta2)
    numGradBias2 = np.zeros_like(bias2)
    
    for i in range(shape(theta1)[0]):
        for j in range(shape(theta1)[1]):
            theta1_pos = np.copy(theta1)
            theta1_neg = np.copy(theta1)
            theta1_pos[i,j] += epsilon
            theta1_neg[i,j] -= epsilon
            numGrad1[i,j]    = (errorFn(X, y, theta1_pos, theta2, bias1, bias2) - \
                                errorFn(X, y, theta1_neg, theta2, bias1, bias2))/2/epsilon
    for i in range(shape(theta2)[0]):
        for j in range(shape(theta2)[1]):
            theta2_pos = np.copy(theta2)
            theta2_neg = np.copy(theta2)
            theta2_pos[i,j] += epsilon
            theta2_neg[i,j] -= epsilon
            numGrad2[i,j]    = (errorFn(X, y, theta1, theta2_pos, bias1, bias2) - \
                                errorFn(X, y, theta1, theta2_neg, bias1, bias2))/2/epsilon
    for i in range(shape(bias1)[0]):
        for j in range(shape(bias1)[1]):
            bias1_pos         = np.copy(bias1)
            bias1_neg         = np.copy(bias1)
            bias1_pos[i,j]   += epsilon
            bias1_neg[i,j]   -= epsilon
            numGradBias1[i,j] = (errorFn(X, y, theta1, theta2, bias1_pos, bias2) - \
                                errorFn(X, y, theta1, theta2, bias1_neg, bias2))/2/epsilon
    for i in range(shape(bias2)[0]):
        for j in range(shape(bias2)[1]):
            bias2_pos         = np.copy(bias2)
            bias2_neg         = np.copy(bias2)
            bias2_pos[i,j]   += epsilon
            bias2_neg[i,j]   -= epsilon
            numGradBias2[i,j] = (errorFn(X, y, theta1, theta2, bias1, bias2_pos) - \
                                errorFn(X, y, theta1, theta2, bias1, bias2_neg))/2/epsilon
    return numGrad1,numGradBias1,numGrad2,numGradBias2

In [13]:
sigmoid = lambda z: 1.0/(1.0+np.exp(-z))
sigmoid_prime = lambda z: multiply(sigmoid(z),(1-sigmoid(z)))
ftanh = lambda z: np.tanh(z)
ftanh_prime = lambda z: 1 - multiply(ftanh(z),ftanh(z))
funny_tanh = lambda z: 1.7159 * np.tanh(2.0/3.0 * z) + .001*z
funny_tanh_prime = lambda z: 1.7159 * 2.0 / 3.0 * (1.0 / multiply(np.cosh(2.0/3.0 * z),np.cosh(2.0/3.0 * z))) + .001
relu = lambda z: multiply(z,(z > 0))
relu_prime = lambda z: z >= 0
leaky_relu = lambda z: np.maximum(.1*z, z)
leaky_relu_prime = lambda z: 1*(z>=0) + .1*(z<0)

act_funcs = {'sigmoid': (sigmoid, sigmoid_prime),
             'ftanh': (ftanh, ftanh_prime),
             'funny_tanh': (funny_tanh, funny_tanh_prime),
             'relu': (relu,relu_prime),
             'leaky_relu': (leaky_relu,leaky_relu_prime)}